In [3]:
# --- FIRST CELL: INSTALL DEPENDENCIES ---

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install git+https://github.com/openai/shap-e
!pip install trimesh pygltflib
!apt-get install -y blender
!pip install bpy
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install gradio
!pip install shap_e # Install the shap_e package
!pip install numpy==1.23.5 # Reinstall numpy==1.23.5 to ensure correct version


Looking in indexes: https://download.pytorch.org/whl/cu118
  Cloning https://github.com/openai/shap-e to /tmp/pip-req-build-90tj84pr
  Running command git clone --filter=blob:none --quiet https://github.com/openai/shap-e /tmp/pip-req-build-90tj84pr
  Resolved https://github.com/openai/shap-e to commit 50131012ee11c9d2617f3886c10f000d3c7a3b43
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-psokvj21/clip_a2439d95826b475fa414323bd1a1d870
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-psokvj21/clip_a2439d95826b475fa414323bd1a1d870
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 101.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing in

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-oa_eeltu/clip_5748e35342b945e7b4aac748bad34ea9
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-oa_eeltu/clip_5748e35342b945e7b4aac748bad34ea9
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray 2025.3.1 requires numpy>=1.24, but you have numpy 1.23.5 which is 

In [1]:
# --- STEP 1: IMPORTS AND SETUP ---

import os
import sys
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import decode_latent_mesh
import trimesh
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
output_dir = '/content/drive/MyDrive/3D_Outputs'
os.makedirs(output_dir, exist_ok=True)
print(f"✅ Outputs will be saved to: {output_dir}")

# Check GPU
if torch.cuda.is_available():
    print(f"🎉 GPU: {torch.cuda.get_device_name(0)}")
    device = torch.device('cuda')
else:
    print("❌ No GPU detected. Change runtime to GPU.")
    device = torch.device('cpu')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Outputs will be saved to: /content/drive/MyDrive/3D_Outputs
🎉 GPU: Tesla T4


In [2]:

# --- STEP 2: LOAD SHAP-E MODELS ---

print("⏳ Loading Shap-E models...")
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))
print("✅ Models loaded successfully!")


⏳ Loading Shap-E models...


/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/shap_e/models/nn/checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/shap_e/models/download.py:136: FutureWarning: Yo

  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

/usr/local/lib/python3.11/dist-packages/shap_e/models/download.py:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(path, map_location=device)


✅ Models loaded successfully!


In [3]:

import gradio as gr
import os
import torch
import trimesh
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import decode_latent_mesh

# Model setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

output_dir = '/content/drive/MyDrive/3D_Outputs'
os.makedirs(output_dir, exist_ok=True)

def generate_mesh(prompt, guidance_scale, steps, format):
    latents = sample_latents(
        batch_size=1,
        model=model,
        diffusion=diffusion,
        guidance_scale=guidance_scale,
        model_kwargs=dict(texts=[prompt]),
        progress=True,
        clip_denoised=True,
        use_fp16=True,
        use_karras=True,
        karras_steps=steps,
        sigma_min=1e-3,
        sigma_max=160,
        s_churn=0,
    )

    # Decode the latent into mesh
    mesh = decode_latent_mesh(xm, latents[0]).tri_mesh()
    mesh = trimesh.Trimesh(vertices=mesh.verts, faces=mesh.faces)

    # Save to file
    filename = f"{prompt.replace(' ', '_')}.{format}"
    path = os.path.join(output_dir, filename)

    if format == "glb":
        mesh.export(path, file_type="glb")
    elif format == "obj":
        mesh.export(path, file_type="obj")
    elif format == "ply":
        mesh.export(path, file_type="ply")
    else:
        raise ValueError("Unsupported format")

    return path

# Gradio UI
gr.Interface(
    fn=generate_mesh,
    inputs=[
        gr.Textbox(label="Prompt", placeholder="e.g., a flying elephant"),
        gr.Slider(5, 20, value=16.5, step=0.5, label="Guidance Scale"),
        gr.Slider(20, 100, value=64, step=1, label="Steps"),
        gr.Radio(["glb", "obj", "ply"], label="Export Format", value="glb")
    ],
    outputs=gr.File(label="Download 3D Model"),
    title="🧠 Text-to-3D Generator (No Blender)",
    description="Generate a 3D model from text and export it without Blender."
).launch(share=True)



/usr/local/lib/python3.11/dist-packages/shap_e/models/download.py:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(path, map_location=device)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c292f78f9079d54132.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
